<a href="https://colab.research.google.com/github/k-washi/3d-Bonet-Boost/blob/master/BoNet_S3DIS_starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3D - BoNetのスタート

データセットは、[Stanford 2D-3D-Semantics Dataset](http://buildingparser.stanford.edu/dataset.html)

h5データへの変更は、[pqhieu/jsis3d](https://github.com/pqhieu/jsis3d/blob/master/scripts/prepare_h5.py)にて提供されている。
また、データセットのh5 versionは、以下の3D-BoNetのリポジトリからダウンロードできる。


# 3D-BoNetのインストール
[Learning Object Bounding Boxes for 3D Instance Segmentation on Point Clouds](https://arxiv.org/abs/1906.01140)

https://github.com/Yang7879/3D-BoNet
---



In [0]:
!rm -rf 3D-BoNet

In [2]:
!git clone https://github.com/Yang7879/3D-BoNet.git

Cloning into '3D-BoNet'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 198 (delta 22), reused 2 (delta 0), pack-reused 151
Receiving objects: 100% (198/198), 71.74 MiB | 9.12 MiB/s, done.
Resolving deltas: 100% (84/84), done.


# 補助パッケージの取得

https://github.com/k-washi/3d-Bonet-Boost

In [0]:
!rm -rf 3d-Bonet-Boost

In [4]:
!git clone https://github.com/k-washi/3d-Bonet-Boost.git

Cloning into '3d-Bonet-Boost'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 36 (delta 17), reused 20 (delta 8), pack-reused 0
Unpacking objects: 100% (36/36), done.


#python ライブラリの確認

tensorflow 1.4

In [4]:
%tensorflow_version 1.x
!pip show tensorflow

Name: tensorflow
Version: 1.4.0
Summary: TensorFlow helps the tensors flow
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: opensource@google.com
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: numpy, tensorflow-tensorboard, six, protobuf, wheel, enum34
Required-by: stable-baselines, magenta, fancyimpute


もし、versionが1.4出ない場合は、変更
もし、RESTAER RUNTIMEが出たら再起動

# データセットのダウンロード

Google Driveにデータセットを置くと仮定

/content/drive/My Drive/data/Data_S3DIS.zip

In [0]:
from google.colab import drive
drive.mount('/content/drive')

データを解凍

In [0]:
!apt-get -y install p7zip-full

In [0]:
!mkdir s3dis
!unzip /content/drive/My\ Drive/data/Data_S3DIS.zip -d s3dis > NILL

# 3D-BoNet: Compile tensorflow operation by cuda 

In [0]:
!cp 3d-Bonet-Boost/cu_compile.py 3D-BoNet/tf_ops/

コンパイルに必要なコマンドを出力

In [13]:
!nvcc --version
!which nvcc

!ls -l /usr/lib/x86_64-linux-gnu/libcudnn.so*
!ls /usr/lib64-nvidia/libcuda*

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130
/usr/local/cuda/bin/nvcc
lrwxrwxrwx 1 root root        29 Dec  5 22:54 /usr/lib/x86_64-linux-gnu/libcudnn.so -> /etc/alternatives/libcudnn_so
lrwxrwxrwx 1 root root        17 Oct 27 21:49 /usr/lib/x86_64-linux-gnu/libcudnn.so.7 -> libcudnn.so.7.6.5
-rw-r--r-- 1 root root 428711256 Oct 27 21:49 /usr/lib/x86_64-linux-gnu/libcudnn.so.7.6.5
/usr/lib64-nvidia/libcuda.so	/usr/lib64-nvidia/libcuda.so.418.67
/usr/lib64-nvidia/libcuda.so.1


In [9]:
!ls /usr/local/cuda-10.1/

bin	extras	 libnsight  nsightee_plugins  README   src	version.txt
compat	include  libnvvp    nvml	      samples  targets
doc	lib64	 LICENSE    nvvm	      share    tools


compileに必要なcudaディレクトリとGC上で実行することを引数で与える

In [11]:
!python 3D-BoNet/tf_ops/cu_compile.py --cuda /usr/local/cuda-10.1 --googleColab

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "/usr/lib/python3.6/imp.py", line 243, in load_module
    return load_dynamic(name, filename, file)
  File "/usr/lib/python3.6/imp.py", line 343, in load_dynamic
    return _load(spec)
ImportError: libcublas.so.8.0: cannot open shared object file: No such file or directory

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "3D-BoNet/tf_o

上のpythonの出力コマンドをコピーして実行 (2020/3/9時点では、以下のコマンド)

In [0]:
!/usr/local/cuda/bin/nvcc 3D-BoNet/tf_ops/../tf_ops/grouping/tf_grouping_g.cu -o 3D-BoNet/tf_ops/../tf_ops/grouping/tf_grouping_g.cu.o -c -O2 -DGOOGLE_CUDA=1 -x cu -Xcompiler -fPIC
!/usr/local/cuda/bin/nvcc 3D-BoNet/tf_ops/../tf_ops/sampling/tf_sampling_g.cu -o 3D-BoNet/tf_ops/../tf_ops/sampling/tf_sampling_g.cu.o -c -O2 -DGOOGLE_CUDA=1 -x cu -Xcompiler -fPIC
!g++ -std=c++11 3D-BoNet/tf_ops/../tf_ops/grouping/tf_grouping.cpp 3D-BoNet/tf_ops/../tf_ops/grouping/tf_grouping_g.cu.o -o 3D-BoNet/tf_ops/../tf_ops/grouping/tf_grouping_so.so -shared -fPIC -I /usr/local/lib/python3.6/dist-packages/tensorflow/include -I /usr/local/cuda/include -I /usr/local/lib/python3.6/dist-packages/tensorflow/include/external/nsync/public -L /usr/local/lib/python3.6/dist-packages/tensorflow -ltensorflow_framework -lcudart -lcuda -L /usr/local/cuda/lib64 -O2 -D_GLIBCXX_USE_CXX11_ABI=0
!g++ -std=c++11 3D-BoNet/tf_ops/../tf_ops/sampling/tf_sampling.cpp 3D-BoNet/tf_ops/../tf_ops/sampling/tf_sampling_g.cu.o -o 3D-BoNet/tf_ops/../tf_ops/sampling/tf_sampling_so.so -shared -fPIC -I /usr/local/lib/python3.6/dist-packages/tensorflow/include -I /usr/local/cuda/include -I /usr/local/lib/python3.6/dist-packages/tensorflow/include/external/nsync/public -L /usr/local/lib/python3.6/dist-packages/tensorflow -ltensorflow_framework -lcudart -lcuda -L /usr/local/cuda/lib64 -O2 -D_GLIBCXX_USE_CXX11_ABI=0
!g++ -std=c++11 3D-BoNet/tf_ops/../tf_ops/interpolation/tf_interpolate.cpp -o 3D-BoNet/tf_ops/../tf_ops/interpolation/tf_interpolate_so.so -shared -fPIC -I /usr/local/lib/python3.6/dist-packages/tensorflow/include -I /usr/local/cuda/include -I /usr/local/lib/python3.6/dist-packages/tensorflow/include/external/nsync/public -L /usr/local/lib/python3.6/dist-packages/tensorflow -ltensorflow_framework -lcudart -lcuda -L /usr/local/cuda/lib64 -O2 -D_GLIBCXX_USE_CXX11_ABI=0

# 3D-BoNet の学習を実行する

In [0]:
import sys
sys.path.append('/content/3D-BoNet/')

In [3]:
import os
import tensorflow as tf

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = '0'  ## specify the GPU to use

from main_3D_BoNet import BoNet
from helper_data_s3dis import Data_Configs as Data_Configs

ImportError: ignored

In [0]:
configs = Data_Configs()
net = BoNet(configs = configs)
net.creat_folders(name='log', re_train=False)

In [0]:
net.build_graph()

In [0]:
tf.summary.merge_all()